In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# Define dictionaries for column names and types
PROCEDURE_COLUMNS = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}
SERVICE_COLUMNS = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}
DATE_COLUMNS = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}
BED_TYPES = {
    "AREA MEDICA ADULTO CUIDADOS BASICOS": "Basicos",
    "AREA MEDICA ADULTO CUIDADOS MEDIOS": "Medias",
    "AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS": "Medias",
    "AREA MÉDICA": "Medias",
    "AREA QUIRÚRGICA": "Medias",
    "CIRUGÍA CARDIOVASCULAR": "Medias",
    "UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO)": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS ADULTO": "UCI",
    "UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA": "UCI",
    "UNIDAD DE RECUPERACIÓN DE PABELLONES (CENTRAL Y CMA)": "AMBULATORIO",
    "UNIDAD DE TRATAMIENTO INTERMEDIO (UTI) (INDIFERENCIADO) ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTO INTERMEDIO MEDICINA ADULTO": "UTI",
    "UNIDAD DE TRATAMIENTOS INTERMEDIOS MEDICINA": "UTI",
}

# Define relevant diagnoses
RELEVANT_DIAGNOSES = [
    "C33",
    "C34.0",
    "C34.1",
    "C34.2",
    "C34.3",
    "C38.1",
    "C38.4",
    "C45.0",
    "C78.0",
    "C78.2",
    "D14.3",
    "D38.1",
    "E84.8",
    "I05.1",
    "I08.0",
    "I08.1",
    "I34.0",
    "I35.0",
    "I35.1",
    "I35.2",
    "I42.0",
    "I45.6",
    "I47.2",
    "I49.5",
    "I71.0",
    "I71.2",
    "J39.8",
    "J47",
    "J67.9",
    "J84.1",
    "J84.8",
    "J84.9",
    "J86.0",
    "J86.9",
    "J90",
    "J93.1",
    "J95.5",
    "J98.0",
    "M34.8",
    "Q21.1",
    "Q23.1",
    "Q67.6",
    "T82.0",
    "T82.1",
    "Z45.0",
]

# Load data
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=PROCEDURE_COLUMNS)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_9372\754381329.py:105: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=PROCEDURE_COLUMNS)


In [3]:
# Filter for Torax hospital
df_torax = df_procesada.query("COD_HOSPITAL == 112103").reset_index()

# Melt service and date columns
service_long = pd.melt(
    df_torax,
    id_vars=["index", "DIAGNOSTICO1", "ANIO_EGRESO"],
    value_vars=SERVICE_COLUMNS,
    var_name="service_type",
    value_name="service",
)
date_long = pd.melt(
    df_torax,
    id_vars=["index"],
    value_vars=DATE_COLUMNS,
    var_name="date_type",
    value_name="date",
)

# Merge service and date data
patient_journey = pd.merge(date_long, service_long, how="inner", left_index=True, right_index=True)
patient_journey = patient_journey.drop(columns=["index_y"])
patient_journey = patient_journey.rename(columns={"index_x": "index"})

# Convert service type to categorical
patient_journey["service_type"] = pd.Categorical(
    patient_journey["service_type"], categories=SERVICE_COLUMNS.keys(), ordered=True
)

# Sort and clean data
patient_journey = (
    patient_journey.sort_values(["index", "service_type"]).dropna().reset_index(drop=True)
)
patient_journey["date"] = pd.to_datetime(patient_journey["date"])

# Calculate length of stay
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

# Fix negative and zero length of stay
negative_indices = patient_journey[patient_journey["length_of_stay"] < pd.Timedelta(0)].index + 1
patient_journey.loc[negative_indices, "date"] += pd.offsets.DateOffset(years=1)
patient_journey["length_of_stay"] = patient_journey.groupby("index")["date"].diff().shift(-1)

zero_indices = patient_journey[patient_journey["length_of_stay"] == pd.Timedelta(0)].index
patient_journey.loc[zero_indices, "length_of_stay"] += pd.Timedelta(days=1)

# Map bed types
patient_journey["service"] = patient_journey["service"].replace(BED_TYPES)

# Summarize length of stay by diagnosis, bed type, and year
summary = (
    patient_journey.groupby(["ANIO_EGRESO", "DIAGNOSTICO1", "service"])["length_of_stay"]
    .sum()
    .unstack()
)
summary_pct = summary.apply(lambda x: x / summary.sum(axis=1), axis=0)
summary_total = pd.concat([summary, summary_pct], axis=1).reset_index()

# Filter for relevant diagnoses
summary_relevant = summary_total.query("DIAGNOSTICO1.isin(@RELEVANT_DIAGNOSES)")

# Save to Excel
with pd.ExcelWriter("../data/interim/resumen_ocupacion_por_diagnostico.xlsx") as writer:
    summary_relevant.to_excel(writer, sheet_name="mas_relevantes", index=False)
    summary_total.to_excel(writer, sheet_name="todos", index=False)